In [ ]:
from scapy.all import *

import random
import string
import time
from datetime import datetime
from tqdm import tqdm
def randStr(chars = string.ascii_uppercase + string.digits, lower=0, upper=100):
    if upper<0:
        upper = 0
    padding = random.randint(lower, upper)
    return ''.join(random.choice(chars) for _ in range(padding))



filters = []
#filters = ['DHCP', 'DNS'] # protocols to exclude

max_len = 1500 # can choose: 1500 (typical MTU) or 65000 (maximum possible length)
trigger = 100000 # update value
min_padding = 1
small_padding = 100 
large_padding = 1024 

min_size = 0 
large_size = 100

In [2]:
import platform
if platform.system() == 'Windows':
    fs = "\\"
else:
    fs = "/"
print(platform.system())

Windows


In [3]:
def byte_udp(input_file, output_file, lower=0, upper=100, debug = False, max_size = 65000):
    start = time.time()
    print("Start Time\n{}".format(datetime.now()))

    input_pcap_size = int(os.path.getsize(input_file) / (1024*1024))
    print("Reading {}MB from {}".format(input_pcap_size, input_file))
    read = 0
    manipulated = 0
    with PcapReader(input_file) as pr:
        print("Manipulating...")
        new_packets = list([])
        for pkt in pr:
            read += 1
            if ((read%trigger) == 0):
                print("...read {} packets...".format(int(read)))
            if 'error' in pkt.summary():
                new_packets.append(pkt)

            elif 'TCP' in pkt.summary():
                # TCP PACKETS are EXCLUDED
                new_packets.append(pkt)

            elif 'UDP' in pkt.summary():
                #check for filters
                if len(filters)>0:
                    if any(f in pkt.summary() for f in filters):
                        continue
                manipulated += 1
                padding = randStr(lower=lower, upper=min((max_size - pkt.len), upper))



                new_pkt = (pkt['Ether'] / padding)

                if 'IPv6' in pkt.summary():
                    del new_pkt['IPv6'].len
                    del new_pkt['IPv6'].chksum
                else:
                    del new_pkt['IP'].len
                    del new_pkt['IP'].chksum
                del new_pkt['UDP'].len
                del new_pkt['UDP'].chksum
                new_pkt = Ether(new_pkt.build())
                new_pkt.time = pkt.time
                new_packets.append(new_pkt)
                
                if debug:
                    if pkt.len>10:
                        print("Old Pkt Len: {}\tUDP len: {}".format(pkt.len, pkt['UDP'].len))
                        print("New Pkt Len: {}\tUDP len: {}".format(new_pkt.len, new_pkt['UDP'].len))
                        print("Padding length: {}".format(len(padding)))

            else:
                new_packets.append(pkt)

    print("Total packets read: {}".format(read))


    print("Writing {} packets (of which {} manipulated)...".format(len(new_packets), manipulated))
    wrpcap(output_file, new_packets)

    end = time.time() - start
    print("End Time:\n{}".format(datetime.now()))
    print("(total running time: {:5f}s)".format(end))

    output_pcap_size = int(os.path.getsize(output_file) / (1024*1024))
    diff = output_pcap_size - input_pcap_size
    print("Wrote a total of {}MB (diff: {}MB) to file: {}".format(output_pcap_size, diff, output_file))
    return new_packets

In [4]:
def byte_tcp(input_file, output_file, lower=0, upper=100, debug = False, max_size = 65000):
    start = time.time()
    print("Start Time\n{}".format(datetime.now()))

    input_pcap_size = int(os.path.getsize(input_file) / (1024*1024))
    print("Reading {}MB from {}".format(input_pcap_size, input_file))
    read = 0
    manipulated = 0
    with PcapReader(input_file) as pr:
        print("Manipulating...")
        new_packets = list([])
        for pkt in pr:
            read += 1
            if ((read%trigger) == 0):
                print("...read {} packets...".format(int(read)))
            if 'error' in pkt.summary():
                new_packets.append(pkt)

            elif 'UDP' in pkt.summary():
                # UDP PACKETS are EXCLUDED
                new_packets.append(pkt)

            elif 'TCP' in pkt.summary():
                if "P" not in pkt['TCP'].flags: #only append data to PUSH type TCP communications
                    new_packets.append(pkt)
                    continue
                #check for filters
                if len(filters)>0:
                    if any(f in pkt.summary() for f in filters):
                        new_packets.append(pkt)
                        continue
                manipulated += 1
                padding = randStr(lower=lower, upper=min((max_size - pkt.len), upper))



                new_pkt = (pkt['Ether'] / padding)

                if 'IPv6' in pkt.summary():
                    del new_pkt['IPv6'].len
                    del new_pkt['IPv6'].chksum
                else:
                    del new_pkt['IP'].len
                    del new_pkt['IP'].chksum
                del new_pkt['TCP'].chksum
                new_pkt = Ether(new_pkt.build())
                new_pkt.time = pkt.time
                new_packets.append(new_pkt)
                
                if debug:
                    if pkt.len>10:
                        print("Old Pkt Len: {}\tTCP len: {}".format(pkt.len, pkt['TCP'].len))
                        print("New Pkt Len: {}\tTCP len: {}".format(new_pkt.len, new_pkt['TCP'].len))
                        print("Padding length: {}".format(len(padding)))

            else:
                new_packets.append(pkt)

    print("Total packets read: {}".format(read))


    print("Writing {} packets (of which {} manipulated)...".format(len(new_packets), manipulated))
    wrpcap(output_file, new_packets)

    end = time.time() - start
    print("End Time:\n{}".format(datetime.now()))
    print("(total running time: {:5f}s)".format(end))

    output_pcap_size = int(os.path.getsize(output_file) / (1024*1024))
    diff = output_pcap_size - input_pcap_size
    print("Wrote a total of {}MB (diff: {}MB) to file: {}".format(output_pcap_size, diff, output_file))
    return new_packets

In [5]:
# input_folder = "/Datasets/NIDS-Datasets/raw/MCP/Malicious/original/"
input_folder = "snippet/original/"
output_folder = "snippet/adversarial/"

In [6]:
for malware in tqdm(os.listdir(input_folder)):
    print(malware)
    for original_pcap in tqdm(os.listdir(input_folder + fs + malware)):
        
        # ATTACKING small tcp!
        attack = "-Ts"
        n = original_pcap.replace(".pcap", attack+".pcap")
        input_file = input_folder + malware + fs + original_pcap
        output_file = output_folder + malware + fs + n
        
        input_pcap_size = int(os.path.getsize(input_file) / (1024*1024))
        if (min_size <= input_pcap_size < large_size): 
            new_packets = byte_tcp(input_file, output_file, lower = min_padding, upper = small_padding)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

trickster



  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Start Time
2024-06-21 18:06:45.281170
Reading 6MB from snippet/original/trickster\2017_08_03_trickster2.pcap
Manipulating...
Total packets read: 37328
Writing 37328 packets (of which 8718 manipulated)...



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:15<00:00, 15.28s/it]

End Time:
2024-06-21 18:07:00.546995
(total running time: 15.265825s)
Wrote a total of 6MB (diff: 0MB) to file: snippet/adversarial/trickster\2017_08_03_trickster2-Ts.pcap


In [7]:
## This is if you want to apply larger perturbations (note that "lower" and "upper" are different)

for malware in tqdm(os.listdir(input_folder)):
    print(malware)
    for original_pcap in tqdm(os.listdir(input_folder + fs + malware)):
        
        # ATTACKING big tcp!
        attack = "-Tb"
        n = original_pcap.replace(".pcap", attack+".pcap")
        input_file = input_folder + malware + fs + original_pcap
        output_file = output_folder + malware + fs + n
        
        input_pcap_size = int(os.path.getsize(input_file) / (1024*1024))
        if (min_size <= input_pcap_size < large_size): 
            new_packets = byte_tcp(input_file, output_file, lower = small_padding, upper = large_padding)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

trickster



  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Start Time
2024-06-21 18:07:00.575294
Reading 6MB from snippet/original/trickster\2017_08_03_trickster2.pcap
Manipulating...
Total packets read: 37328
Writing 37328 packets (of which 8718 manipulated)...



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:17<00:00, 17.12s/it]

End Time:
2024-06-21 18:07:17.683845
(total running time: 17.108551s)
Wrote a total of 11MB (diff: 5MB) to file: snippet/adversarial/trickster\2017_08_03_trickster2-Tb.pcap


In [8]:
## This is if you want to apply larger perturbations (note that "lower" and "upper" are different)

for malware in tqdm(os.listdir(input_folder)):
    print(malware)
    for original_pcap in tqdm(os.listdir(input_folder + fs + malware)):
        
        # ATTACKING big udp!
        attack = "-Ub"
        n = original_pcap.replace(".pcap", attack+".pcap")
        input_file = input_folder + malware + fs + original_pcap
        output_file = output_folder + malware + fs + n
        
        input_pcap_size = int(os.path.getsize(input_file) / (1024*1024))
        if (min_size <= input_pcap_size < large_size): 
            new_packets = byte_udp(input_file, output_file, lower = small_padding, upper = large_padding)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

trickster



  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Start Time
2024-06-21 18:07:17.702221
Reading 6MB from snippet/original/trickster\2017_08_03_trickster2.pcap
Manipulating...
Total packets read: 37328
Writing 37328 packets (of which 1253 manipulated)...



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:10<00:00, 10.73s/it]

End Time:
2024-06-21 18:07:28.429432
(total running time: 10.727211s)
Wrote a total of 7MB (diff: 1MB) to file: snippet/adversarial/trickster\2017_08_03_trickster2-Ub.pcap


In [9]:
for malware in tqdm(os.listdir(input_folder)):
    print(malware)
    for original_pcap in tqdm(os.listdir(input_folder + fs + malware)):
        
        # ATTACKING small udp!
        attack = "-Us"
        n = original_pcap.replace(".pcap", attack+".pcap")
        input_file = input_folder + malware + fs + original_pcap
        output_file = output_folder + malware + fs + n
        
        input_pcap_size = int(os.path.getsize(input_file) / (1024*1024))
        if (min_size <= input_pcap_size < large_size): 
            new_packets = byte_udp(input_file, output_file, lower = min_padding, upper = small_padding)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

trickster



  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Start Time
2024-06-21 18:07:28.449433
Reading 6MB from snippet/original/trickster\2017_08_03_trickster2.pcap
Manipulating...
Total packets read: 37328
Writing 37328 packets (of which 1253 manipulated)...



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:10<00:00, 10.68s/it]

End Time:
2024-06-21 18:07:39.119960
(total running time: 10.670527s)
Wrote a total of 6MB (diff: 0MB) to file: snippet/adversarial/trickster\2017_08_03_trickster2-Us.pcap
